## Installation des librairies

In [2]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.0 MB/s eta 0:00:00


## Importation des librairies

In [3]:
import pandas as pd
import json
from elasticsearch import Elasticsearch

## Connexion à Elasticsearch

In [ ]:
es = Elasticsearch(
    hosts=['https://674b6f7a3a534dcdb8ac1c94995bd9bc.us-central1.gcp.cloud.es.io:443'],
    http_auth=('elastic', 'QZAa7KvM2BPw7p4W2wMWmEsx')
)

# Tester la connexion :
print(es.info())

<ipython-input-3-a5c1c33defd3>:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


{'name': 'instance-0000000001', 'cluster_name': '674b6f7a3a534dcdb8ac1c94995bd9bc', 'cluster_uuid': 'V5wknLdpQRWHFaONBev7Dw', 'version': {'number': '8.13.4', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'da95df118650b55a500dcc181889ac35c6d8da7c', 'build_date': '2024-05-06T22:04:45.107454559Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}




---


# Exercice 1 : `Indexation`

In [ ]:
###############################################   # Exercice 1: Indexation

# Énoncé: Indexez un nouveau document dans l'index ecommerce.

doc = {
    "product": "Chaussures de piscine",
    "price": 19.99,
    "currency": "EUR",
    "quantity": 10,
    "brand": "SportyCo",
    "timestamp": "2023-08-29T12:01:00"
}

response = es.index(index="ecommerce", document=doc)
print(response)

{'_index': 'ecommerce', '_id': '8_lF2Y8B44bp-FDQUru2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


In [ ]:
response

ObjectApiResponse({'_index': 'ecommerce', '_id': '8_lF2Y8B44bp-FDQUru2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})



---


# Exercice 2 : `Recherche simple`

In [ ]:
###############################################   Exercice 2: Recherche simple

## Énoncé: Effectuez une recherche pour tous les produits dont le nom contient le mot "sport".

response = es.search(index="ecommerce", body={
    "query": {
        "match": {
            "product": "piscine sport"
        }
    },
    "explain": "true"
})

print(response)

In [ ]:
response

ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.6931471, 'hits': [{'_shard': '[ecommerce][0]', '_node': 'EaHnNTS3QNmDnn98aciMFQ', '_index': 'ecommerce', '_id': '-eQEYI0BeKZ3WHn9us4x', '_score': 0.6931471, '_source': {'product': 'Chaussures de sport', 'price': 109.99, 'currency': 'EUR', 'quantity': 20, 'brand': 'SportyCo', 'timestamp': '2023-08-29T12:01:00'}, '_explanation': {'value': 0.6931471, 'description': 'weight(product:sport in 0) [PerFieldSimilarity], result of:', 'details': [{'value': 0.6931471, 'description': 'score(freq=1.0), computed as boost * idf * tf from:', 'details': [{'value': 2.2, 'description': 'boost', 'details': []}, {'value': 0.6931472, 'description': 'idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:', 'details': [{'value': 1, 'description': 'n, number of documents containing term', 'details': []}, {'value': 2, 'descripti

In [ ]:
# Get the key "hits" and understand the model
hits = response['hits']['hits']
hits

[{'_shard': '[ecommerce][0]',
  '_node': 'EaHnNTS3QNmDnn98aciMFQ',
  '_index': 'ecommerce',
  '_id': '-eQEYI0BeKZ3WHn9us4x',
  '_score': 0.9808291,
  '_source': {'product': 'Chaussures de sport',
   'price': 109.99,
   'currency': 'EUR',
   'quantity': 20,
   'brand': 'SportyCo',
   'timestamp': '2023-08-29T12:01:00'},
  '_explanation': {'value': 0.9808291,
   'description': 'sum of:',
   'details': [{'value': 0.9808291,
     'description': 'weight(product:sport in 0) [PerFieldSimilarity], result of:',
     'details': [{'value': 0.9808291,
       'description': 'score(freq=1.0), computed as boost * idf * tf from:',
       'details': [{'value': 2.2, 'description': 'boost', 'details': []},
        {'value': 0.98082924,
         'description': 'idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:',
         'details': [{'value': 1,
           'description': 'n, number of documents containing term',
           'details': []},
          {'value': 3,
           'description': 'N, total 

In [ ]:
# Get source information
source_list = [hit['_source'] for hit in hits]
df = pd.json_normalize(source_list)
print(df)

                 product   price currency  quantity     brand  \
0    Chaussures de sport  109.99      EUR        20  SportyCo   
1  Chaussures de piscine   19.99      EUR        10  SportyCo   

             timestamp  
0  2023-08-29T12:01:00  
1  2023-08-29T12:01:00  




---


# Exercice 3 : `Mise à jour`

In [ ]:
###############################################   Exercice 3: Mise à jour

# Énoncé: Mettez à jour le document que vous avez indexé précédemment
# (utilisez l'ID du document retourné lors de l'indexation) pour changer la colonne 'quantity' à 95.

# Correction: (Notez que vous devez remplacer YOUR_DOCUMENT_ID par l'ID réel)

response = es.update(index="ecommerce", id="ueRGWo0BeKZ3WHn9XJAy", body={
    "doc": {
        "quantity": 95
    }
})

print(response)

{'_index': 'ecommerce', '_id': 'ueRGWo0BeKZ3WHn9XJAy', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}


In [ ]:
response = es.search(index="ecommerce", body={
    "query": {
        "match_all": {
        }
    }
})

print(response)

{'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'ecommerce', '_id': 'aNIBYI0Bxbyce13nAIFG', '_score': 1.0, '_source': {'product': 'Chaussures de course', 'price': 109.99, 'currency': 'EUR', 'quantity': 20, 'brand': 'SportyCo', 'timestamp': '2023-08-29T12:01:00'}}, {'_index': 'ecommerce', '_id': '-eQEYI0BeKZ3WHn9us4x', '_score': 1.0, '_source': {'product': 'Chaussures de sport', 'price': 109.99, 'currency': 'EUR', 'quantity': 20, 'brand': 'SportyCo', 'timestamp': '2023-08-29T12:01:00'}}, {'_index': 'ecommerce', '_id': 'eNIGYI0Bxbyce13nnIH3', '_score': 1.0, '_source': {'product': 'Chaussures de piscine', 'price': 19.99, 'currency': 'EUR', 'quantity': 10, 'brand': 'SportyCo', 'timestamp': '2023-08-29T12:01:00'}}]}}


In [ ]:
# Get source information
hits = response['hits']['hits']
source_list = [hit['_source'] for hit in hits]
df = pd.json_normalize(source_list)
print(df)

                 product   price currency  quantity     brand  \
0   Chaussures de course  109.99      EUR        20  SportyCo   
1    Chaussures de sport  109.99      EUR        20  SportyCo   
2  Chaussures de piscine   19.99      EUR        10  SportyCo   

             timestamp  
0  2023-08-29T12:01:00  
1  2023-08-29T12:01:00  
2  2023-08-29T12:01:00  




---


# Exercice 4 : `Agrégations`

In [ ]:
###############################################   Exercice 4: Agrégations

# Énoncé: Obtenez le prix moyen de tous les produits dans l'index ecommerce.

# Correction:
response = es.search(index="ecommerce", body={
    "size": 0,
    "aggs": {
        "avg_price": {
            "avg": {
                "field": "price"
            }
        }
    }
})
avg_price = response['aggregations']['avg_price']['value']
print(f"Prix moyen: {avg_price} EUR")


Prix moyen: 59.9900016784668 EUR




---


# Exercice 5 : `Suppression`

In [ ]:
####################################################    Exercice 5: Suppression
##Correction: (Remplacez YOUR_DOCUMENT_ID par l'ID réel)

response = es.delete(index="ecommerce", id="ueRGWo0BeKZ3WHn9XJAy")
print(response)



---


# Exercice 7 : `Recherche multiple`

In [ ]:
###############################################   Exercice 7: Recherche multiple

## Énoncé: Effectuez une recherche pour tous les produits dont le nom contient le mot "sport".

response = es.search(index="ecommerce", body={
    "query": {
        "multi_match": {
            "query": "sport piscine",
            "fields": ["product"]
        }
    },
    "explain": "true"
})

print(response)

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 0.9808291, 'hits': [{'_shard': '[ecommerce][0]', '_node': 'wjujGcMfQjiCbF7c0g-pSQ', '_index': 'ecommerce', '_id': '-eQEYI0BeKZ3WHn9us4x', '_score': 0.9808291, '_source': {'product': 'Chaussures de sport', 'price': 109.99, 'currency': 'EUR', 'quantity': 20, 'brand': 'SportyCo', 'timestamp': '2023-08-29T12:01:00'}, '_explanation': {'value': 0.9808291, 'description': 'sum of:', 'details': [{'value': 0.9808291, 'description': 'weight(product:sport in 0) [PerFieldSimilarity], result of:', 'details': [{'value': 0.9808291, 'description': 'score(freq=1.0), computed as boost * idf * tf from:', 'details': [{'value': 2.2, 'description': 'boost', 'details': []}, {'value': 0.98082924, 'description': 'idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:', 'details': [{'value': 1, 'description': 'n, number of documents containing term

In [ ]:
# Get the key "hits" and understand the model
hits = response['hits']['hits']
hits

[{'_shard': '[ecommerce][0]',
  '_node': 'wjujGcMfQjiCbF7c0g-pSQ',
  '_index': 'ecommerce',
  '_id': '-eQEYI0BeKZ3WHn9us4x',
  '_score': 0.9808291,
  '_source': {'product': 'Chaussures de sport',
   'price': 109.99,
   'currency': 'EUR',
   'quantity': 20,
   'brand': 'SportyCo',
   'timestamp': '2023-08-29T12:01:00'},
  '_explanation': {'value': 0.9808291,
   'description': 'sum of:',
   'details': [{'value': 0.9808291,
     'description': 'weight(product:sport in 0) [PerFieldSimilarity], result of:',
     'details': [{'value': 0.9808291,
       'description': 'score(freq=1.0), computed as boost * idf * tf from:',
       'details': [{'value': 2.2, 'description': 'boost', 'details': []},
        {'value': 0.98082924,
         'description': 'idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:',
         'details': [{'value': 1,
           'description': 'n, number of documents containing term',
           'details': []},
          {'value': 3,
           'description': 'N, total 

In [ ]:
# Get source information
source_list = [hit['_source'] for hit in hits]
df = pd.json_normalize(source_list)
print(df)

                 product   price currency  quantity     brand  \
0    Chaussures de sport  109.99      EUR        20  SportyCo   
1  Chaussures de piscine   19.99      EUR        10  SportyCo   

             timestamp  
0  2023-08-29T12:01:00  
1  2023-08-29T12:01:00  
